In [ ]:
import crepe
import numpy as np
import librosa
import os

In [1]:
from html.parser import HTMLParser

new_track = False
class MyHTMLParser(HTMLParser):
    def __init__ (self):
        HTMLParser.__init__(self)
        self.data = []

    def handle_starttag(self, tag, attrs):
        new_track = True
        if tag == "track-wrap":
            for key, val in attrs:
                if key == "title":
                    self.data.append(val)
        if tag == "video":
            self.data.append("video")
        if tag == "source":
            for key, val in attrs:
                if key == "src":
                    self.data.append(os.path.basename(val))

    def handle_endtag(self, tag):
        if tag == "track-wrap" or tag == "video":
            new_track = False

    def handle_data(self, data):
        pass
    
    def get_data (self):
        return zip(self.data, self.data[1:] + self.data[:1])

In [ ]:
import os
import re
from os import listdir
from os.path import isdir, join, splitext, exists

root = '/Akamai/voice/data/'
alg = 'pitches-raw/crepe'
collections = ['Scherbaum Mshavanadze/']
for coll in collections:
    songdirs = [d for d in listdir(join(root,coll)) if isdir(join(root, coll, d))]
    for sd in songdirs: 
        files = dict()
        html= open(join(root, coll, sd, 'page.txt')).read()
        parser = MyHTMLParser()
        parser.feed(html)
        files['throat'] = [f[1] for f in \
                           list(filter(lambda x: re.search("Throat microphone [0-9]+", x[0]), parser.get_data()))]
        files['headset'] = [f[1] for f in \
                            list(filter(lambda x: re.search("Headset microphone [0-9]+", x[0]), parser.get_data()))]
        
        for mic in files:
            for f in files[mic]:
                #if re.search('ALRX2M', f) == None:
                 #  continue
                f0dir = join(root, alg, coll, sd)
                os.makedirs(f0dir, exist_ok=True)
                f0file = join(f0dir, splitext(f)[0] + '.txt')
                if exists(f0file):
                    print("Skipping: ", f0file)
                    continue
                data_path = join(root, coll, sd, f[:-3] + 'wav')
                audio, sr = librosa.load(data_path, sr=None)
                time, frequency, confidence, activation = crepe.predict(audio, sr, viterbi=True, verbose=False)
                np.savetxt(f0file, np.column_stack((time, frequency, confidence)), fmt='%6.2f %7.2f %7.4f')
                print("Done: ", f[:-3] + 'wav')

In [2]:
def crepe_calc (data_dir, pitch_dir, from_path):
    y, sr = librosa.load(data_dir + from_path, sr=None)
    time, frequency, confidence, activation = crepe.predict(y, sr, viterbi=True, verbose=False)
    np.savetxt(pitch_dir + from_path[:-3] + 'txt', np.column_stack((time, frequency, confidence)), fmt='%6.2f %7.2f %7.4f')
    print("Done: ", from_path[:-3] + 'wav')

In [ ]:
# Teach Yourself Megrelian Songs
# parent_data_dir = '/Akamai/voice/data/Teach Yourself Megrelian Songs/'
# parent_pitch_dir = '/Akamai/voice/data/pitches-raw/crepe/Teach Yourself Megrelian Songs/'

# for collection in os.listdir(parent_data_dir):
#     if os.path.isdir(f"{parent_data_dir}{collection}"):
#         if collection != 'mp3':
#             parts = []
#             for part in os.listdir(f"{parent_data_dir}{collection}"):
#                 if part[-3:] == 'wav':
#                     parts.append(part)

#             data_dir = parent_data_dir + collection + '/'
#             pitch_dir = parent_pitch_dir + collection + '/'
            
#             for part in parts:
#                 crepe_calc(data_dir, pitch_dir, part)
                
                
# Teach Yourself Gurian Songs
parent_data_dir = '/Akamai/voice/data/Teach Yourself Gurian Songs/'
parent_pitch_dir = '/Akamai/voice/data/pitches-raw/crepe/Teach Yourself Gurian Songs/'

for collection in os.listdir(parent_data_dir):
    if os.path.isdir(f"{parent_data_dir}{collection}"):
        if collection != 'mp3':
            parts = []
            for part in os.listdir(f"{parent_data_dir}{collection}"):
                if part[-3:] == 'wav':
                    parts.append(part)

            data_dir = parent_data_dir + collection + '/'
            pitch_dir = parent_pitch_dir + collection + '/'
            
            for part in parts:
                crepe_calc(data_dir, pitch_dir, part)